In [27]:
import numpy as np
import json
import os
import cv2
import matplotlib.pyplot as plt
# from scipy.spatial import distance
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

In [47]:
HEAD_bb_path = './results/head_bounding_boxes/train_v3.csv' # Hollywoodheads
HEAD_bb_path_2 = './results/head_bounding_boxes/test_v3.csv'
PERSON_bb_dir= './results/person_bounding_boxes/film8/'
ORIGINAL_IMG = './data/head_det_corpus_v3/film8/'
MATCH_OUT_DIR= './results/match/film8/'
if not os.path.exists(MATCH_OUT_DIR):
    os.makedirs(MATCH_OUT_DIR)

In [29]:
heads = open(HEAD_bb_path, 'r').readlines()
heads.extend(open(HEAD_bb_path_2, 'r').readlines())
print(len(heads))

73391


In [83]:
def computeIoH(person_bb, head_bb, epsilon=0.1):
    # intersection over head-box = area of overlap / (area of head-box + epsilon* area of person) => "intersection over union"-like stuff
    headbox_area = (head_bb[2]-head_bb[0])*(head_bb[3]-head_bb[1])
    person_area = (person_bb[2]-person_bb[0])*(person_bb[3]-person_bb[1])
    dx = min(head_bb[2], person_bb[2])-max(head_bb[0], person_bb[0])
    dy = min(head_bb[3], person_bb[3])-max(head_bb[1], person_bb[1])
    overlap_area = -1
    if dx > 0 and dy > 0: # make sure person and head intersects
        overlap_area = dx * dy
    return -overlap_area/(headbox_area+epsilon*person_area)

def center(person_bb, head_bb, distance='euclidean'):
    # compute distance from the two centers
    width_head = head_bb[2]-head_bb[0]
    height_head = head_bb[3]-head_bb[1]
    center_head = np.array([head_bb[0]+width_head/2, head_bb[1]+height_head/2])
    width_person = person_bb[2]-person_bb[0]
    height_person = person_bb[3]-person_bb[3]
    center_person = np.array([person_bb[0]+width_person/2, person_bb[1]+height_person/2])
    return distance

In [91]:
def generateColor():
    color = tuple(np.random.choice(range(256), size=3))
    return tuple(int(c) for c in color)

In [98]:
for filename in os.listdir(PERSON_bb_dir):
    if filename.find('.json') != -1:
        json_data = json.load(open(os.path.join(PERSON_bb_dir, filename)))
        detections = []
        if 'detections' in json_data.keys():
            detections = json_data['detections']
        person_bbs = [det['bbox'] for det in detections if det['class']=='person']
        raw_filename = (PERSON_bb_dir.strip().split('/'))[-2]+ '/' + '.'.join((filename.strip().split('.'))[0:-1])
        head_line = [line for line in heads if line.find(raw_filename)!=-1]
        head_bbs = []
        if len(head_line) > 0 and len(person_bbs) > 0:
            head_bbs = (head_line[0].strip().split('\t'))[1:]
            head_bbs = [[int(head_bbs[i]), int(head_bbs[i+1]), int(head_bbs[i+2]), int(head_bbs[i+3])] for i in range(len(head_bbs)) if i%5==0]
            print(raw_filename,'\nHead\t', head_bbs, '\nPerson\t', person_bbs)
            C = cdist(XA=head_bbs, XB=person_bbs, metric=computeIoH) # maximize
            indices = linear_sum_assignment(C)
#             print("Cost", C[indices[0], indices[1]].sum())
            pair_indices=[(ind1,ind2) for ind1,ind2 in zip(indices[0],indices[1])]            
            img_filename = '.'.join((filename.strip().split('.'))[0:-1]) + '.png'
            image = cv2.imread(os.path.join(ORIGINAL_IMG, img_filename))
            mismatched_head = [i for i in range(len(head_bbs)) if i not in indices[0]]
            mismatched_person = [i for i in range(len(person_bbs)) if i not in indices[1]]
            print(raw_filename)
            for (row_ind, col_ind) in pair_indices:
                if C[row_ind, col_ind] < 0:
#                     print('Head: ', row_ind, head_bbs[row_ind], '\nPerson: ', col_ind, person_bbs[col_ind])
                    color = generateColor()
                    cv2.rectangle(image, (head_bbs[row_ind][0], head_bbs[row_ind][1]), (head_bbs[row_ind][2], head_bbs[row_ind][3]), 
                                  color, 2)
                    cv2.rectangle(image, (person_bbs[col_ind][0], person_bbs[col_ind][1]), (person_bbs[col_ind][2], person_bbs[col_ind][3]), 
                                  color, 1)
            for i in mismatched_head:
                cv2.rectangle(image, (head_bbs[i][0], head_bbs[i][1]), (head_bbs[i][2], head_bbs[i][3]), 
                                  (0,0,255), 2)
            for i in mismatched_person:
                cv2.rectangle(image, (person_bbs[i][0], person_bbs[i][1]), (person_bbs[i][2], person_bbs[i][3]), 
                                  (0,255, 0), 1)
            cv2.imwrite(os.path.join(MATCH_OUT_DIR, img_filename),image)

film8/out374 
Head	 [[343, 25, 485, 214]] 
Person	 [[303, 6, 686, 397]]
film8/out374
film8/out369 
Head	 [[433, 59, 550, 199], [134, 22, 254, 183]] 
Person	 [[26, 33, 328, 401], [298, 57, 704, 402]]
film8/out369
film8/out155 
Head	 [[296, 38, 384, 165]] 
Person	 [[173, 31, 523, 402]]
film8/out155
film8/out261 
Head	 [[568, 322, 657, 405], [166, 305, 249, 394], [401, 304, 483, 403], [536, 287, 579, 337], [209, 285, 241, 316], [359, 281, 393, 316], [524, 274, 550, 301], [640, 272, 680, 318], [406, 272, 431, 301], [293, 272, 324, 306], [180, 272, 209, 303], [246, 271, 276, 307], [490, 266, 513, 292], [323, 264, 354, 297], [81, 259, 110, 290], [322, 249, 344, 270], [585, 243, 606, 265], [347, 242, 357, 257], [118, 241, 130, 257], [275, 240, 289, 256], [204, 239, 218, 255], [614, 236, 635, 259], [645, 227, 671, 254], [544, 225, 562, 246], [524, 223, 537, 239], [536, 222, 550, 239], [147, 220, 177, 253], [577, 217, 591, 234], [389, 211, 401, 228], [690, 199, 716, 228], [28, 199, 67, 238], [3